In [ ]:
class Carta :
    def __init__(self, palo, valor):
        self.palo = palo
        self.valor = valor

class Mazo :
    def __init__(self):
        self.mazo = self.generar_cartas()
        self.tamaño = len(self.mazo)

    def generar_cartas(self) :
        palos = ['Corazones', 'Diamantes', 'Tréboles', 'Picas']
        valores = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']

        mazo = []
        for palo in palos :
            for valor in valores :
                mazo.append(Carta(palo,valor))
    
        return mazo

    def revolver(self) :
        import random
        random.shuffle(self.mazo)

    def repartir(self, jugadores: list, cartas_por_mano: int) :
        num_jugadores = len(jugadores)

        manos = []
        for i in range(0,num_jugadores) :
            mano = []
            for j in range(0,cartas_por_mano) :
                mano.append(self.mazo[i*cartas_por_mano+j])
            manos.append(mano)    

        for i in range(0,num_jugadores) :
            jugadores[i].número = i+1
            jugadores[i].mano = manos[i]

class Jugador :
    def __init__(self, nombre):
        self.nombre = nombre
        self.número = 0
        self.mano = []

    def mostrar_mano(self) :
        mano = []
        for carta in self.mano :
            mano.append(f"{carta.valor} de {carta.palo}")
        # print(mano)
        return mano

In [ ]:
def buscar_flor(jugador: object) :
    valores_imperiales = ['10', 'J', 'Q', 'K', 'A']

    cartas_Corazones = []
    cartas_Diamantes = []
    cartas_Tréboles  = []
    cartas_Picas = []
    for carta in jugador.mano :
        match carta.palo :
            case 'Corazones' :
                cartas_Corazones.append(carta)
            case 'Diamantes' :
                cartas_Diamantes.append(carta)
            case 'Tréboles' :
                cartas_Tréboles.append(carta)
            case 'Picas' :
                cartas_Picas.append(carta)

    conjuntos_cartas = [cartas_Corazones,cartas_Diamantes,cartas_Tréboles,cartas_Picas]

    for conjunto in conjuntos_cartas :
        indicador_palos = 0
        if len(conjunto) == len(jugador.mano) :
            indicador_palos = 1
            break
    
    suma_valores = 0
    for carta in jugador.mano :
        if carta.valor in valores_imperiales :
            suma_valores += 1
    
    if suma_valores + indicador_palos == 6 :
        # print(f"{jugador.nombre} tiene la Flor Imperial.")
        return True
    else :
        # print("Flor Imperial no encontrada.")
        return False

In [ ]:
Juan = Jugador("Juan")
Pedro = Jugador("Pedro")
Pablo = Jugador("Pablo")
Paco = Jugador("Paco")

mazo = Mazo()

In [ ]:
Jugadas_total = 0
Jugadas_imperiales = 0

for i in range(0,1000000) :
    mazo.revolver()
    mazo.repartir([Juan,Pedro,Pablo,Paco],5)

    for jugador in [Juan,Pedro,Pablo,Paco] :
        if buscar_flor(jugador) :
            Jugadas_imperiales += 1
            print(f"{jugador.nombre}: {jugador.mostrar_mano()}")

    Jugadas_total += 1

In [ ]:
Jugadas_imperiales / Jugadas_total * 100